## 1 Functions and module

### 1.1 Modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import math
import seaborn as sns
import re
import scipy.stats as SS
import copy
import scipy
import itertools

### 1.2 Functions

## 2 Input and output address

In [ ]:
raw_input_address = 'Cas12a_Efficiency_final_df.parquet'

In [ ]:
data_dir = 'data/'

In [ ]:
gRNA_combination_address = data_dir+'Efficiency_BT_NormalMethod_KTCas12a_N300_R1000_gRNA_combination_level_summary.csv'
gene_combination_address = data_dir+'Efficiency_BT_NormalMethod_KTCas12a_N300_R1000_gene_combination_level_summary.csv'
gene_combination_unordered_address = data_dir+'Efficiency_BT_NormalMethod_KTCas12a_N300_R1000_gene_combination_unordered_level_summary.csv'

In [ ]:
KT_gRNA_combination_address = data_dir+'Efficiency_BT_NormalMethod_KT_N300_R1000_gRNA_combination_level_summary.csv'
KT_gene_combination_address = data_dir+'Efficiency_BT_NormalMethod_KT_N300_R1000_gene_combination_level_summary.csv'
KT_gene_combination_unordered_address = data_dir+'Efficiency_BT_NormalMethod_KT_N300_R1000_gene_combination_unordered_level_summary.csv'
safe_vs_NT_address = data_dir+'Safe_NT_BT_1000.csv'

In [ ]:
df3_output_address = data_dir+'EfficiencyTable_for_gene_level_tumor_metrics_consider_position.csv'
df4_output_address = data_dir+'EfficiencyTable_for_gene_level_tumor_metrics_ignore_position.csv'
df5_output_address = data_dir+'EfficiencyTable_for_NT_Safe_comparison.csv'

## 3 Data Input and simple QC

### 3.0 Read array reference info

In [198]:
raw_input_df = pd.read_parquet(raw_input_address)

In [199]:
ref_list = ['gRNA_combination', 'gRNA1', 'gRNA2', 'gRNA3','gRNA4', 
            'Guide1', 'Guide2','Guide3', 'Guide4', 'Gene1', 'Gene2', 'Gene3', 'Gene4', 
            'gRNA_combination_unordered', 'gene_combination',
            'gene_combination_unordered', 'Type','Array_category']

In [200]:
ref_df = raw_input_df[ref_list].drop_duplicates()

In [201]:
del raw_input_df

### 3.1 gRNA combination level metrics

In [202]:
df1 = pd.read_csv(gRNA_combination_address)

In [203]:
df1 = df1.merge(ref_df, on = 'gRNA_combination', how ='left')

### 3.3 gene combination level metrics

In [204]:
df3 = pd.read_csv(gene_combination_address)

df3 = df3.merge(ref_df[['gene_combination','gene_combination_unordered']].drop_duplicates(), on = 'gene_combination', how ='left')

In [205]:
df3_KT = pd.read_csv(KT_gene_combination_address)

df3_KT = df3_KT.merge(ref_df[['gene_combination','gene_combination_unordered']].drop_duplicates(), on = 'gene_combination', how ='left')

### 3.4 unordered gene level metrics

In [206]:
df4 = pd.read_csv(gene_combination_unordered_address)
df4_KT = pd.read_csv(KT_gene_combination_unordered_address)

### 3.5 Safe vs NT

In [207]:
df5 = pd.read_csv(safe_vs_NT_address)

-------

## 4 Generate output table for gene_combination_unordered

In [208]:
temp_list = ['gene_combination_unordered','TTN']
query_list = ['LN_mean_relative','95_percentile_relative','TTN_normalized_relative']
trait_list = []
for x in query_list:
    temp = [x,f'{x}_bootstrap_median',f'{x}_bootstrap_mean',f'{x}_97.5P',f'{x}_2.5P',f'{x}_pvalue_twoside',f'{x}_pvalue_twoside_FDR']
    trait_list = trait_list+temp

In [209]:
subset1 = df4[temp_list+trait_list].copy()
subset1['Genotype'] ='KTCas12a'

In [210]:
subset2 = df4_KT[temp_list+trait_list].copy()
subset2['Genotype'] ='KT'

In [211]:
df4_Output = pd.concat([subset1,subset2],ignore_index=True)

In [212]:
df4_Output.to_csv(df4_output_address,index=False)

## 5 Generate output table for gene_combination

In [213]:
temp_list = ['gene_combination','gene_combination_unordered','TTN']
query_list = ['LN_mean_relative','95_percentile_relative','TTN_normalized_relative']
trait_list = []
for x in query_list:
    temp = [x,f'{x}_bootstrap_median',f'{x}_bootstrap_mean',f'{x}_97.5P',f'{x}_2.5P',f'{x}_pvalue_twoside',f'{x}_pvalue_twoside_FDR']
    trait_list = trait_list+temp

In [214]:
subset1 = df3[temp_list+trait_list].copy()
subset1['Genotype'] ='KTCas12a'

In [215]:
subset2 = df3_KT[temp_list+trait_list].copy()
subset2['Genotype'] ='KT'

In [216]:
df3_Output = pd.concat([subset1,subset2],ignore_index=True)

In [217]:
df3_Output.to_csv(df3_output_address,index=False)

## 6 Safe vs NT

In [218]:
df5 = df5.rename(columns = {'Joined_Genes':'gene_combination'})

In [219]:
temp_list = ['gene_combination','TTN']
query_list = ['LN_mean_relative','95_percentile_relative','TTN_normalized_relative']
trait_list = []
for x in query_list:
    temp = [x,f'{x}_bootstrap_median',f'{x}_bootstrap_mean',f'{x}_97.5P',f'{x}_2.5P',f'{x}_pvalue_twoside',f'{x}_pvalue_twoside_FDR']
    trait_list = trait_list+temp

In [220]:
df5_Output =  df5[temp_list+trait_list].copy()

In [221]:
df5_Output

,gene_combination,TTN,LN_mean_relative,LN_mean_relative_bootstrap_median,LN_mean_relative_bootstrap_mean,LN_mean_relative_97.5P,LN_mean_relative_2.5P,LN_mean_relative_pvalue_twoside,LN_mean_relative_pvalue_twoside_FDR,95_percentile_relative,95_percentile_relative_bootstrap_median,95_percentile_relative_bootstrap_mean,95_percentile_relative_97.5P,95_percentile_relative_2.5P,95_percentile_relative_pvalue_twoside,95_percentile_relative_pvalue_twoside_FDR,TTN_normalized_relative,TTN_normalized_relative_bootstrap_median,TTN_normalized_relative_bootstrap_mean,TTN_normalized_relative_97.5P,TTN_normalized_relative_2.5P,TTN_normalized_relative_pvalue_twoside,TTN_normalized_relative_pvalue_twoside_FDR
0,NT_NT_NT_NT,1221.0,1.088016,1.104377,1.089261,1.362105,0.946010,0.366,0.610,1.140602,1.143054,1.123761,1.416279,0.971934,0.252,0.6000,0.943271,0.949236,0.961227,1.114734,0.766923,0.352,0.586667
1,NT_NT_NT_Safe,22872.0,0.970095,0.979831,0.988223,1.015716,0.915247,0.160,0.610,1.000000,0.988080,1.000000,1.068654,0.892590,0.360,0.6000,1.000659,1.006227,1.000000,1.106802,0.920689,0.710,0.862000
2,NT_NT_Safe_Safe,3655.0,0.961164,0.974071,0.981554,1.046597,0.886846,0.310,0.610,0.983682,0.963740,0.977912,1.075362,0.833686,0.316,0.6000,1.081088,1.096800,1.083845,1.284644,0.984968,0.274,0.586667
3,NT_Safe_Safe_Safe,2129.0,1.014429,1.024478,1.007731,1.151034,0.940838,0.892,0.892,1.019300,1.035253,1.000294,1.252564,0.880310,0.998,0.9980,1.000000,1.009486,1.000000,1.213158,0.835770,0.862,0.862000
4,Safe_Safe_Safe_Safe,2345.0,1.000000,1.007936,1.000000,1.123582,0.920674,0.788,0.892,0.963631,0.973963,0.977172,1.180794,0.806564,0.562,0.7025,0.946268,0.960416,0.968343,1.101991,0.827008,0.350,0.586667


In [222]:
df5_Output.to_csv(df5_output_address,index=False)